In [1]:
import pandas as pd 
import numpy as np
df = pd.read_parquet('../scripts/df_merged1.parquet', engine='pyarrow')

In [2]:
df.shape

(5796, 10)

In [3]:
unique_ids = df['objectId'].unique().tolist()

In [4]:
condition = (df['source'] == 0) | (df['dc_sigflux'] == 0)
df['dc_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
df['nr_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

In [5]:
df[df['source'] == 0]

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_weight,nr_weight
2,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.0,0.0
3,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
5,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
6,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59083,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.0,0.0
7,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59083,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5784,ZTF18abxavej,1356446886315015012,2.459111e+06,59105,1,0.001210,0.000079,0.000962,1.113912e-19,0,0.0,0.0
5785,ZTF18abxavej,1356446886315015012,2.459111e+06,59105,2,0.001272,0.000018,0.000994,2.219433e-19,0,0.0,0.0
5787,ZTF18abxavej,1356446886315015012,2.459111e+06,59106,2,0.001272,0.000018,0.000994,2.219433e-19,0,0.0,0.0
5790,ZTF18abxavej,1356446886315015012,2.459111e+06,59108,1,0.001210,0.000079,0.000962,1.113912e-19,0,0.0,0.0


In [6]:
df[['source','dc_weight']].head()

,source,dc_weight
0,1,1.353949e+09
1,1,2.615609e+08
2,0,0.000000e+00
3,0,0.000000e+00
4,1,2.325079e+08


In [108]:
len(unique_ids)

100

In [109]:
import time 
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids}
#print(d)

count = 0

for k in unique_ids:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        #print(len(s), len(s_n1))
        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        #alpha = s[::2] + s[1::2] 
        #print(s)
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

ZTF18acevrat :  [2.72223802e-29 6.60467896e-29 1.72424973e-28 3.45576088e-28
 3.40134284e-28 0.00000000e+00 0.00000000e+00 4.25159071e-28
 1.72261550e-28 1.21136515e-28 2.65502893e-28 0.00000000e+00
 1.65446204e-28 0.00000000e+00 0.00000000e+00 0.00000000e+00
 1.49826529e-28 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 7.52460487e-29 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00] 

ZTF18abnvapv :  [0.00000000e+00 0.00000000e+00 0.00000000e+00 1.23952226e-29
 0.00000000e+00 0.00000000e+00 0.00000000e+00 1.12819566e-29
 0.00000000e+00 0.00000000e+00 1.06286671e-29 0.00000000e+00
 0.00000000e+00 0.00000000e+00 8.87730117e-30 1.42468172e-29
 0.00000000e+00 7.04650499e-30 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
 0.00000000e+00 0.00000000e+00] 

ZTF17aaadyyl :  [0.00000000e+00 0.00000000e+00

In [18]:
(d['ZTF17aaaeclk']<= factor)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True, False, False,  True,  True, False,  True,
        True, False,  True,  True,  True,  True,  True,  True,  True,
       False,  True,  True, False])

In [19]:
d['ZTF17aaaeclk']

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       2.21897294e+00, 1.88563829e-29, 0.00000000e+00, 1.80586019e-01,
       0.00000000e+00, 2.13493190e-29, 0.00000000e+00, 0.00000000e+00,
       7.68288175e+00, 5.32782820e+00, 0.00000000e+00, 6.50042413e-30,
       1.39304523e+02, 0.00000000e+00, 0.00000000e+00, 2.15648677e+02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.23409884e+00,
       0.00000000e+00, 0.00000000e+00, 9.56681501e+00])

In [10]:
Q

,dc_flux
0,0.000286
1,0.000636


In [20]:
df[df['objectId']=='ZTF17aaaeclk']

,objectId,candid,jd,mjd,fid,dc_flux,dc_sigflux,nr_flux,nr_sigflux,source,dc_weight,nr_weight
0,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,1,0.000286,0.000027,0.000286,3.955427e-06,1,1.353949e+09,6.391653e+10
1,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59080,2,0.000636,0.000062,0.000636,1.172011e-05,1,2.615609e+08,7.280090e+09
2,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.000000e+00,0.000000e+00
3,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59081,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
4,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59082,1,0.000286,0.000066,0.000286,3.955427e-06,1,2.325079e+08,6.391653e+10
...,...,...,...,...,...,...,...,...,...,...,...,...
57,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59108,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
58,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59109,1,0.000275,0.000119,0.000286,5.598796e-20,0,0.000000e+00,0.000000e+00
59,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59109,2,0.000679,0.000133,0.000636,1.137121e-19,0,0.000000e+00,0.000000e+00
60,ZTF17aaaeclk,1356301332615015001,2.459111e+06,59110,1,0.000357,0.000010,0.000286,3.955427e-06,1,9.836937e+09,6.391653e+10


In [14]:
alpha

array([1.        , 1.        , 0.        , 0.        , 1.        ,
       1.        , 0.        , 0.        , 0.8657912 , 0.8657912 ,
       1.68378527, 1.68378527, 0.75325967, 0.75325967, 1.46160216,
       1.46160216, 0.68641158, 0.68641158, 1.6263246 , 1.6263246 ,
       0.6566954 , 0.6566954 , 0.        , 0.        , 0.67570275,
       0.67570275, 0.70926008, 0.70926008, 1.        , 1.        ,
       0.69205042, 0.69205042, 1.01079325, 1.01079325, 0.        ,
       0.        , 1.        , 1.        , 1.07928014, 1.07928014,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.26119821,
       1.26119821, 0.        , 0.        , 0.        , 0.        ,
       1.32646186, 1.32646186])

In [21]:
1.32646186*np.array((0.000286,0.000636))

array([0.00037937, 0.00084363])

#

# with l>0

In [ ]:
import time 
start_time = time.time()



m=0
factor = 2*m+1 + 3*np.sqrt(2*(2*m+1))
l=1
Q = df[['dc_flux']].iloc[:2*(m+1)]#.values # j:j+m ?

d = {k: 0 for k in unique_ids[0:1]}
#print(d)

count = 0

for k in unique_ids[0:100]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn
        n = len(T)
        zeros = np.zeros(n, dtype=float)
        s_1 = zeros.copy()
        s_2 = zeros.copy()
        f = T['dc_flux'].values
        w = T['dc_weight'].values
        
        for j in range(0,m+1): 
            #f = T['dc_flux'].values[j*2:] # * 2 !!! for r and g right ? 
            #w = T['dc_sigflux_weight'].values[j*2:]
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            s_1[:n-j*2] += (f[j*2:]*h*w[j*2:])
            
            s_2[:n-j*2] += (h**2 * w[j*2:])


            #s = np.where(s_d == 0, 0, s_n / s_d)
        s_n = s_1[::2] + s_1[1::2]  # this needs to optimizate with new variables ! 
        s_d = s_2[::2] + s_2[1::2] 
        
        mask_no_0 = (s_d != 0)
        s = np.zeros_like(s_d, dtype=float)

        #print(len(s), len(s_n1))
        s[mask_no_0] = s_n[mask_no_0] / s_d[mask_no_0] # # Perform division only where s_d(i) is not zero
         
            
        #alpha = s[::2] + s[1::2] 
        #print(s)
        alpha = np.repeat(s, 2) # duplicate alpha for each value (one for r and second for g)
        
        dd = zeros.copy()
        
        for j in range(0,m+1):
            h = np.tile(Q[j*2:j*2+2], (len(f[j*2:]) // 2, 1)).ravel() # array of h for r and g successive for the vectorisation

            dd[:n-j*2] += ((f[j*2:] - alpha[:n-j*2] * h)**2) * w[j*2:] # :-j*2 it works but not for 0 

        d[k] = dd[::2] + dd[1::2]
            
            
    if (d[k] <= factor).all():
        print(k, ": ", d[k],"\n")
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")

In [203]:
# Define the size of each chunk

m =1
chunk_size = 2 * (m + 1)
print(chunk_size)
# Calculate the number of chunks
num_chunks = len(df[['dc_flux']]) // chunk_size
print(num_chunks)
# Split the DataFrame into chunks and store them in an array
Q = [df[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

if m>0: 
    for j in range(1, m+1):
        print(j)
        Q.extend([df[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
#Q

4
1449
1


In [207]:
print((len(Q)+1)/len(df))

0.5


In [41]:
j=0
np.tile(Q[3][j*2:j*2+2], (4 // 2, 1)).ravel()

array([0.00027479, 0.00067947, 0.00027479, 0.00067947])

In [174]:
import pandas as pd

values = list(range(8))  

# Duplicate each value
duplicated_values = [val for val in values for _ in range(2)]


df2 = pd.DataFrame({'dc_flux': duplicated_values})

print(df2)

    dc_flux
0         0
1         0
2         1
3         1
4         2
5         2
6         3
7         3
8         4
9         4
10        5
11        5
12        6
13        6
14        7
15        7


In [201]:
# Define the size of each chunk
m=1
chunk_size = 2 * (m + 1)

# Calculate the number of chunks
num_chunks = len(df2[['dc_flux']]) // chunk_size

# Split the DataFrame into chunks and store them in an array
Q = [df2[['dc_flux']].iloc[i*chunk_size : (i+1)*chunk_size] for i in range(num_chunks)]

Q

[   dc_flux
 0        0
 1        0
 2        1
 3        1,
    dc_flux
 4        2
 5        2
 6        3
 7        3,
     dc_flux
 8         4
 9         4
 10        5
 11        5,
     dc_flux
 12        6
 13        6
 14        7
 15        7]

In [202]:
for j in range(1, m+1):
    Q.extend([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
    #print([df2[['dc_flux']].iloc[i*chunk_size +j*2 : (i+1)*chunk_size+j*2] for i in range(num_chunks-1)])
############################### importante !!! 
Q



[   dc_flux
 0        0
 1        0
 2        1
 3        1,
    dc_flux
 4        2
 5        2
 6        3
 7        3,
     dc_flux
 8         4
 9         4
 10        5
 11        5,
     dc_flux
 12        6
 13        6
 14        7
 15        7,
    dc_flux
 2        1
 3        1
 4        2
 5        2,
    dc_flux
 6        3
 7        3
 8        4
 9        4,
     dc_flux
 10        5
 11        5
 12        6
 13        6]

In [132]:
for i in range(len(Q)-1):
    #print([Q[i][2*m:],Q[i+1][:2*m]])
    print(i)
    #print([Q[i].iloc[2*m:], Q[i+1].iloc[:2*m]])
    Q.append(pd.concat([Q[i].iloc[2*m-2:], Q[i+1].iloc[:2*m-2]]))
    Q.append(pd.concat([Q[i].iloc[2*m:], Q[i+1].iloc[:2*m]]))


0
1
2


In [133]:
Q

[   dc_flux
 0        0
 1        0
 2        1
 3        1
 4        2
 5        2,
     dc_flux
 6         3
 7         3
 8         4
 9         4
 10        5
 11        5,
     dc_flux
 12        6
 13        6
 14        7
 15        7
 16        8
 17        8,
     dc_flux
 18        9
 19        9
 20       10
 21       10
 22       11
 23       11,
    dc_flux
 2        1
 3        1
 4        2
 5        2
 6        3
 7        3,
     dc_flux
 8         4
 9         4
 10        5
 11        5
 12        6
 13        6,
     dc_flux
 14        7
 15        7
 16        8
 17        8
 18        9
 19        9]

#

#

#

#

In [105]:
d

{'ZTF17aaaeclk': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.21897294e+00, 1.88563829e-29, 0.00000000e+00, 1.80586019e-01,
        0.00000000e+00, 2.13493190e-29, 0.00000000e+00, 0.00000000e+00,
        7.68288175e+00, 5.32782820e+00, 0.00000000e+00, 6.50042413e-30,
        1.39304523e+02, 0.00000000e+00, 0.00000000e+00, 2.15648677e+02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 8.23409884e+00,
        0.00000000e+00, 0.00000000e+00, 9.56681501e+00]),
 'ZTF17aaarimh': array([1.03200771e+02, 0.00000000e+00, 0.00000000e+00, 1.04255940e+02,
        5.98643906e+00, 0.00000000e+00, 0.00000000e+00, 5.41807322e+01,
        5.20258029e+02, 4.63097414e+02, 2.39603720e-29, 1.14760345e+02,
        0.00000000e+00, 1.73092479e-29, 6.51668370e+01, 1.92845006e+02,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        1.73757194e+01, 0.00000000e+00, 0.0000

In [110]:
list_of_dicts = []

# Iterate through the data dictionary and create dictionaries for each object ID
for object_id, distances in d.items():
    for distance in distances:
        list_of_dicts.append({'objectId': object_id, 'distance': distance})

# Create DataFrame from the list of dictionaries
pdf = pd.DataFrame(list_of_dicts)

# Display the DataFrame
print(pdf)

          objectId      distance
0     ZTF17aaaeclk  0.000000e+00
1     ZTF17aaaeclk  0.000000e+00
2     ZTF17aaaeclk  0.000000e+00
3     ZTF17aaaeclk  0.000000e+00
4     ZTF17aaaeclk  2.218973e+00
...            ...           ...
2893  ZTF18abxavej  4.215970e-29
2894  ZTF18abxavej  1.320325e+01
2895  ZTF18abxavej  0.000000e+00
2896  ZTF18abxavej  3.100433e+02
2897  ZTF18abxavej  3.083399e+02

[2898 rows x 2 columns]


In [113]:
len(df)/2898

2.0

In [117]:
pdf['is_distance_gt_factor'] = pdf['distance'] <= factor

In [118]:
pdf

,objectId,distance,is_distance_gt_factor
0,ZTF17aaaeclk,0.000000e+00,True
1,ZTF17aaaeclk,0.000000e+00,True
2,ZTF17aaaeclk,0.000000e+00,True
3,ZTF17aaaeclk,0.000000e+00,True
4,ZTF17aaaeclk,2.218973e+00,True
...,...,...,...
2893,ZTF18abxavej,4.215970e-29,True
2894,ZTF18abxavej,1.320325e+01,False
2895,ZTF18abxavej,0.000000e+00,True
2896,ZTF18abxavej,3.100433e+02,False


# Trash

In [ ]:
df[df['objectId'] =='ZTF18acevrat' ]

In [ ]:

import timeit

def approach1():
    condition = df['source'] == 0
    df['dc_sigflux_weight'] = np.where(condition, 0, 1 / (df['dc_sigflux'] ** 2))
    df['nr_sigflux_weight'] = np.where(condition, 0, 1 / (df['nr_sigflux'] ** 2))

def approach2():
    df['dc_sigflux_weight'] = 1 / (df['dc_sigflux'] ** 2)
    df['nr_sigflux_weight'] = 1 / (df['nr_sigflux'] ** 2)
    df['dc_sigflux_weight'] = df['dc_sigflux_weight'] * (df['source'] != 0)
    df['nr_sigflux_weight'] = df['nr_sigflux_weight'] * (df['source'] != 0)

time_approach2 = timeit.timeit(approach2, number=10)
time_approach1 = timeit.timeit(approach1, number=10)

print("1 execution time:", time_approach1)
print("2 execution time:", time_approach2)


In [ ]:
m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?

T = {unique_ids.index(k): df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:1]}
d = {unique_ids.index(k): np.zeros(int(len(T[k])/2), dtype=np.float64) for k in unique_ids[0:1]}

for j in range(1,m+1): 
    #h = Q.values     
    
    
            for filt in np.unique(T[k]['fid']):
                mask = T[k]['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[k][mask]['dc_flux'].values
                w = T[k][mask]['dc_sigflux_weight'].values

                s_n = (f[-1+j:]*h[j-1] * w[-1+j:])
                s_d = (h[j-1]**2 * w[-1+j:])

                alpha = s_n / s_d
                #print("alpha", alpha)
                
                d[k] += ((f[-1+j:] - alpha * h[j-1])**2) * w[-1+j:]

                #break
            
if (d[k] <= factor).any():
        #print(d)
        count +=1
        #print(count, k)
        
print(count)

In [ ]:
import time 
start_time = time.time()



m=1
factor = 2*m -1 + 3*np.sqrt(2*(2*m -1))
l=1
Q = df[['fid', 'dc_flux']].iloc[:2*m]#.values # j:j+m ?
#print(Q)

#T = {k: df[df['objectId'] == k][['fid', 'dc_flux', 'dc_sigflux_weight']] for k in unique_ids[0:100]}
d0 = {k: 0 for k in unique_ids[0:10]}
#print(d)

count = 0

for k in unique_ids[0:10]:
    
    T = df[df['objectId'] == k][['fid','dc_flux', 'dc_sigflux_weight']]
    for l in [0]:
        #d[k]=0
        # we don't use it rn 
        for j in range(0,m): 
            
            alpha = 0
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                s_n = (f[j:]*h[j] * w[j:])
                s_d = (h[j]**2 * w[j:])
                s = np.where(s_d == 0, 0, s_n / s_d)

                alpha += s
                #print("alpha", alpha)
            for filt in np.unique(T['fid']):
                mask = T['fid'] == filt
                h = Q[Q['fid'] == filt ]['dc_flux'].values     
                f = T[mask]['dc_flux'].values
                w = T[mask]['dc_sigflux_weight'].values

                d0[k] += ((f[j:] - alpha * h[j])**2) * w[j:]
               #print()
               #print(d)

            #break
            
    if (d0[k] <= factor).all():
        print(d0[k],k)
        count +=1
        #print(count, k)
        

    #else:
        
        #print(d[k],k)
        
print(count)

end_time = time.time()

# Compute the elapsed time
elapsed_time = end_time - start_time

print("Elapsed time:", elapsed_time, "seconds")
       